In [1]:
import sys
import os
import gc
gc.collect()
os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1, 2'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # or any {'0', '1', '2'}

import cv2
import numpy as np
import traceback
sys.path.insert(1, '../../Src/')
from utils import vision
from utils import camera
from utils import trackerSort
from utils import trackerNorfair

COLOR_BOUNDINGBOX = (255, 0, 0)
THICKNESS_BOUNDINGBOX = 2

from models.mediapipe import model_mediapipe
model = model_mediapipe.MODEL_mediapipe()

#tracker = trackerSort.Tracker()
tracker = trackerNorfair.Tracker()

   

cap = camera.ParallelCamera(0).start()
my_fps = vision.FPS(100)
my_fps_model = vision.FPS(100)

try:

  while True:
    my_fps.start()

    # Read image
    success, image = cap.read()  
    if not success:
      print("Ignoring empty camera frame.")
      break
    
    # Preprocessing
    image = cv2.resize(image,(500,500))
    image = cv2.flip(image.copy(), 1)
    #image = np.array(np.flip(image,1),dtype=np.uint8)

    
    # Model Inference
    my_fps_model.start()  
    image.flags.writeable = False
    boxes, scores= model.predict(image)
    image.flags.writeable = True  
    my_fps_model.update()
    
    # Model Tracking
    boxes_tracker = tracker.predict(boxes)
    
    # Drawing bbox    
    image = vision.draw_boundingboxes(image,boxes_tracker,COLOR_BOUNDINGBOX,THICKNESS_BOUNDINGBOX,if_wh=False,is_track=True)

    # Calculate FPS
    my_fps.update()

    # Show image
    image = vision.put_text(image,"FPS      : "+str(np.round(my_fps.fps(),3)),pos =(20,20))
    image = vision.put_text(image,"FPS model: "+str(np.round(my_fps_model.fps(),3)),pos =(20,50))
    cv2.imshow('Face Detection Project',image)
    gc.collect()

    # Close window with ESC
    if cv2.waitKey(5) & 0xFF == 27:
      break

except Exception as e:
    cap.release()
    cv2.destroyAllWindows()
    import sys
    sys.exc_info()
    print(traceback.format_exc())  # or: traceback.print_exc()
    
    
# DestroyWindows and clean memory
cap.release()
cv2.destroyAllWindows()
model.close()


/media/cristian/12FF1F6D0CD48422/Tesis/env_3.10_com/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/media/cristian/12FF1F6D0CD48422/Tesis/env_3.10_com/lib/python3.10/site-packages/sklearn/utils/linear_assignment_.py:18: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  warnings.warn(


type id <class 'int'>
start webcam id: 0...


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


start webcam finished!
